In [93]:
from keras.models import Model, Sequential, Model
from keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D, Activation, Lambda
from keras.optimizers import SGD
from keras.metrics import cosine_proximity

In [94]:
def baseNetwork(inp_dim):
    embeddingSize = 128
    inputShape = (1, inp_dim, 1)
    kernelSizeConv1 = (1,10)
    kernelSizeConv2 = (1,5)
    mPool_PoolSize1 = (1, 100)
    mPool_PoolSize2 = (1, 5)
    mPoolStride1 = (1, 100)
    mPoolStride2 = (1, 4)
    optLayersConv1 = 5 ## No of output layers in conv 1
    optLayersConv2 = 8 ## No of output layers in conv 2
    
    convNet = Sequential()
    ## Conv layer 1
    convNet.add(Conv2D(optLayersConv1, kernel_size = kernelSizeConv1, input_shape = inputShape))
    convNet.add(MaxPooling2D(pool_size = mPool_PoolSize1, strides = mPoolStride1))
    convNet.add(Activation('relu'))
    ## Conv layer 2
    convNet.add(Conv2D(optLayersConv2, kernel_size = kernelSizeConv2))
    convNet.add(MaxPooling2D(pool_size = mPool_PoolSize2, strides = mPoolStride2))
    convNet.add(Activation('relu'))
    
    convNet.add(Flatten())
    convNet.add(Dense(embeddingSize)) 
    
    return convNet

In [95]:
import numpy as np
totalSize = 5000
batchSize = 50
inputDimension = 5000
questions = np.random.random((totalSize,inputDimension)).reshape(totalSize, 1, inputDimension, 1)
answers = np.random.random((totalSize,inputDimension)).reshape(totalSize, 1, inputDimension, 1)
labels = [1 if p>0.5 else -1 for p in np.random.random(totalSize)]
#print(questions[0:10])
#print(answers[0:10])
#print(labels[0:10])
print(questions.shape)
print(questions.shape[2:])
inputQ = Input(shape=questions.shape[1:])
inputA = Input(shape=answers.shape[1:])
baseNw = baseNetwork(inputDimension)
convQues = baseNw(inputQ)
convAns = baseNw(inputA)
#conv.summary()

(5000, 1, 5000, 1)
(5000, 1)


In [96]:
from keras import backend as K
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)
#print([inputA, inputQ])
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([convQues, convAns])
#print(distance)

In [97]:
model = Model(input=[inputQ, inputA], output=distance)
#rms = RMSprop()
model.compile(loss = cosine_proximity, optimizer=SGD(lr=0.01, momentum=0.05))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_21 (InputLayer)            (None, 1, 5000, 1)    0                                            
____________________________________________________________________________________________________
input_22 (InputLayer)            (None, 1, 5000, 1)    0                                            
____________________________________________________________________________________________________
sequential_37 (Sequential)       (None, 128)           11655       input_21[0][0]                   
                                                                   input_22[0][0]                   
____________________________________________________________________________________________________
lambda_8 (Lambda)                (None, 1)             0           sequential_37[1][0]     

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`
  """Entry point for launching an IPython kernel.


In [ ]:
noEpochs = 10
model.fit([questions, answers], labels, batch_size=batchSize, epochs=noEpochs)

Epoch 1/10
5000/5000 [==============================] - 9s - loss: -0.0088     
Epoch 2/10
5000/5000 [==============================] - 8s - loss: -0.0088     
Epoch 3/10
5000/5000 [==============================] - 8s - loss: -0.0088     
Epoch 4/10
 700/5000 [===>..........................] - ETA: 7s - loss: 0.0086  